In [139]:
import numpy as np
import matplotlib.pyplot as plt
import sjoert.stellar
import pandas as pd
import os
from tqdm import tqdm
from cleaning_functions import q_cuts,field_check,filter_split,flux_unc_val,clean_data
from astropy import coordinates as coord

sjoertpath = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\Sjoert_Flares'

In [120]:
PATH = os.path.normpath(os.getcwd() + os.sep + os.pardir)
DATAPATH = os.path.join(PATH,'Data')
CODEPATH = os.path.join(PATH,'Code')
columns = ['sindex', 'field', 'ccdid', 'qid', 'filter', 'pid', 'infobitssci', 'sciinpseeing', 'scibckgnd', 'scisigpix', 'zpmaginpsci', 'zpmaginpsciunc', 'zpmaginpscirms', 'clrcoeff', 'clrcoeffunc', 'ncalmatches', 'exptime', 'adpctdif1', 'adpctdif2', 'diffmaglim', 'zpdiff', 'programid', 'jd', 'rfid', 'forcediffimflux', 'forcediffimfluxunc', 'forcediffimsnr', 'forcediffimchisq', 'forcediffimfluxap', 'forcediffimfluxuncap', 'forcediffimsnrap', 'aperturecorr', 'dnearestrefsrc', 'nearestrefmag', 'nearestrefmagunc', 'nearestrefchi', 'nearestrefsharp', 'refjdstart', 'refjdend', 'procstatus']
dtypes = [(columns[x],float) for x in range(len(columns))]
dtypes[4] = ('filter',r'U8')

test_path = os.path.join(DATAPATH,r"18")
test_path = os.path.join(sjoertpath,'ZTF19abiptrq')
print(test_path)
test_lc = pd.DataFrame(np.genfromtxt(os.path.join(test_path,os.listdir(test_path)[0]),skip_header=53,dtype=dtypes))
test_lc


C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\Sjoert_Flares\ZTF19abiptrq


,sindex,field,ccdid,qid,filter,pid,infobitssci,sciinpseeing,scibckgnd,scisigpix,...,forcediffimsnrap,aperturecorr,dnearestrefsrc,nearestrefmag,nearestrefmagunc,nearestrefchi,nearestrefsharp,refjdstart,refjdend,procstatus
0,0.0,599.0,9.0,4.0,ZTF_g,5.484412e+11,0.0,2.1651,83.2122,9.80441,...,NaN,1.156464,NaN,NaN,NaN,NaN,NaN,2.458276e+06,2.458348e+06,NaN
1,1.0,599.0,9.0,4.0,ZTF_r,5.514343e+11,0.0,1.8785,154.5100,8.24687,...,0.042127,1.072413,0.298118,19.066,0.098,5.831,0.534,2.458285e+06,2.458367e+06,0.0
2,2.0,599.0,9.0,4.0,ZTF_r,5.564411e+11,0.0,3.4364,142.3760,5.10206,...,-0.109556,1.225016,0.298118,19.066,0.098,5.831,0.534,2.458285e+06,2.458367e+06,0.0
3,3.0,599.0,9.0,4.0,ZTF_r,5.594201e+11,0.0,1.6128,143.0450,5.28476,...,2.152287,1.070403,0.298118,19.066,0.098,5.831,0.534,2.458285e+06,2.458367e+06,0.0
4,4.0,599.0,9.0,4.0,ZTF_g,5.594857e+11,0.0,2.7016,77.8977,7.32948,...,NaN,1.162553,NaN,NaN,NaN,NaN,NaN,2.458276e+06,2.458348e+06,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868,1868.0,599.0,9.0,4.0,ZTF_r,2.467227e+12,0.0,1.7547,174.9880,8.67583,...,-2.494993,1.044787,0.298118,19.066,0.098,5.831,0.534,2.458285e+06,2.458367e+06,0.0
1869,1869.0,599.0,9.0,4.0,ZTF_r,2.469226e+12,0.0,2.2862,176.2400,7.12707,...,-0.748987,1.067333,0.298118,19.066,0.098,5.831,0.534,2.458285e+06,2.458367e+06,0.0
1870,1870.0,599.0,9.0,4.0,ZTF_g,2.469279e+12,0.0,2.5212,94.9387,6.81256,...,NaN,1.120983,NaN,NaN,NaN,NaN,NaN,2.458276e+06,2.458348e+06,NaN
1871,1871.0,599.0,9.0,4.0,ZTF_g,2.474249e+12,0.0,2.9565,95.6172,4.82302,...,NaN,1.132190,NaN,NaN,NaN,NaN,NaN,2.458276e+06,2.458348e+06,NaN


In [154]:
## Code to remove all clean files and clean logs from the sjoert_flares directory.
# for folder in os.listdir(sjoertpath)[::-1]:
#     if 'ZTF' in folder:
#         folderpath = os.path.join(sjoertpath,folder)
#         for file in os.listdir(folderpath):
#             if 'clean' in file:
#                 os.remove(os.path.join(folderpath,file))

In [123]:
iok = q_cuts(test_lc) * field_check(test_lc)
idx_bad = np.where(np.invert(iok))[0]

data_ok = test_lc[iok]

no_ir_idx = np.where(data_ok['filter'] != 'ZTF_i')[0]
data_ok_no_ir = data_ok.copy(deep=True)
data_ok_no_ir = data_ok_no_ir.iloc[no_ir_idx]
norm_jd_full = np.array(data_ok_no_ir['jd'] - data_ok_no_ir['jd'].iloc[0])

clean_data_full = pd.DataFrame({"time":norm_jd_full,"flux":data_ok_no_ir['forcediffimflux'].values,
                                # "flux_unc":np.zeros(data_ok_no_ir.shape[0]),
                                "zeropoint":data_ok_no_ir['zpdiff'].values,
                                "filter" :data_ok_no_ir['filter'].values})

# clean_data_full['flux_unc'].iloc[np.where(clean_data_full['filter'] == 'ZTF_g')[0]] = 1
clean_data_full.sort_values('filter',inplace=True)

new_unc_tot = []
for filter,filter_data in filter_split(data_ok).items():
    new_unc = np.array(flux_unc_val(filter_data))
    if filter != 'ZTF_i':
        new_unc_tot.append(new_unc)
new_unc_tot = np.concatenate(new_unc_tot).ravel()
clean_data_full['flux_unc'] = new_unc_tot

# for item in os.listdir(test_path):
#     if 'ZTF_r' in item:
#         r_path = os.path.join(test_path,item)
#     if 'ZTF_g' in item:
#         g_path = os.path.join(test_path,item)
#     else:
#         continue

# rdata = pd.read_csv(r_path,delimiter=' ')
# gdata = pd.read_csv(g_path,delimiter=' ')
# max(gdata['flux_unc'].values - clean_data_full['flux_unc'].loc[clean_data_full['filter']=='ZTF_g'].values)
clean_data_full['filter'].unique() # so it works

array(['ZTF_r'], dtype=object)

In [ ]:
def clean_data(data,ZTF_ID,filename=None,savepath=None,verbose=False):
    iok = q_cuts(data) #very first quality check, mask array of good data points.
    data_ok = data[iok]
    logdict = {} #dictionary that will form the log.json file

    filters = np.unique(data_ok['filter'])

    if iok.sum() == 0: #this might occur, this prevents an error
        print(f"{ZTF_ID}: no viable data found in the batch request. Proceeding to next file.")
        return 
    else:
        if 'ZTF_g' not in filters or 'ZTF_r' not in filters:
            print(ZTF_ID,": There is no viable ZTF_g or there is no viable ZTF_r data. Proceeding cleaning.")

    for filter,filtermask in filter_split(data_ok,asmasks=True).items():
        logdict[filter] = {}
        data_ok_filter = data_ok[filtermask]
        fields,field_counts = np.unique(data_ok_filter['field'],return_counts=True)
        primary_field = [i == np.max(field_counts) for i in field_counts]

        for i,fid in enumerate(fields):
            field_mask = data_ok_filter['field'] == fid
            data_ok_filter_field = data_ok_filter[field_mask]
            iok_filter_field = iok * filtermask * field_mask 

            if primary_field[i]:
                if iok_filter_field.sum() == 0:
                    print(f"{ZTF_ID}: no viable data found in the primary field. Proceeding to next file.")
                    return 
                else:
                    if 'ZTF_g' not in data_ok_filter_field['filter'].unique() or 'ZTF_r' not in data_ok_filter_field['filter'].unique():
                        print(ZTF_ID,": There is no viable ZTF_g or there is no viable ZTF_r data in the primary field. Proceeding cleaning.")
                        for filter,filter_data in filter_split(data_ok).items():
                            new_unc = np.array(flux_unc_val(filter_data))

                            norm_jd = np.array(filter_data['jd'] - filter_data['jd'].iloc[0])
                            clean_data = pd.DataFrame({"time":norm_jd,
                                                        "flux":filter_data['forcediffimflux'].values,
                                                        "flux_unc":new_unc,
                                                        "zeropoint":filter_data['zpdiff'].values})
                            if savepath != None:
                                assert filename != None, 'Filename should be specified if savepath is specified.'
                                savestring = os.path.join(savepath,"clean_"+filter+"_"+filename)
                                clean_data.to_csv(savestring,sep=' ',index=False)
                            else:
                                if verbose:
                                    print(filter,clean_data)
                                    print(20*'--')
                    return

            # logdict[filter][fid] = {'ZTF_ID':ZTF_ID,'zeropoint':data_ok_filter_field['zpdiff'],'primary_field':primary_field[i],'removed_cleaning':np.sum(iok_filter_field)},'chi2':15,'chi2_after':14}
            # logdict[filter]['chi2'] = ...
            # logdict[filter]['chi2_after'] = ...





    idx_bad = np.where(np.invert(iok))[0]

    no_ir_idx = np.where(data_ok['filter'] != 'ZTF_i')[0]
    data_ok_no_ir = data_ok.copy(deep=True)
    data_ok_no_ir = data_ok_no_ir.iloc[no_ir_idx]
    try:
        norm_jd_full = np.array(data_ok_no_ir['jd'] - data_ok_no_ir['jd'].iloc[0])
    except:
        if iok.sum() == 0:
            print(f"{savepath}: no viable data found. Proceeding.")
            return 
        else:
            if 'ZTF_g' not in data_ok['filter'].unique() or 'ZTF_r' not in data_ok['filter'].unique():
                print("There is no viable ZTF_g or there is no viable ZTF_r data. Proceeding cleaning.")
                for filter,filter_data in filter_split(data_ok).items():
                    new_unc = np.array(flux_unc_val(filter_data))

                    norm_jd = np.array(filter_data['jd'] - filter_data['jd'].iloc[0])
                    clean_data = pd.DataFrame({"time":norm_jd,
                                                "flux":filter_data['forcediffimflux'].values,
                                                "flux_unc":new_unc,
                                                "zeropoint":filter_data['zpdiff'].values})
                    if savepath != None:
                        assert filename != None
                        savestring = os.path.join(savepath,"clean_"+filter+"_"+filename)
                        clean_data.to_csv(savestring,sep=' ',index=False)
                    else:
                        if verbose:
                            print(filter,clean_data)
                            print(20*'--')
                return


    clean_data_full = pd.DataFrame({"time":norm_jd_full,"flux":data_ok_no_ir['forcediffimflux'].values,
                                # "flux_unc":np.zeros(data_ok_no_ir.shape[0]),
                                "zeropoint":data_ok_no_ir['zpdiff'].values,
                                "filter" :data_ok_no_ir['filter'].values})

    if savepath != None:
        pd.DataFrame({"bad_idx":[idx_bad],"median_chi_before":np.median(data['forcediffimchisq']),"median_chi_after":np.median(data_ok['forcediffimchisq'])}).to_csv(\
            os.path.join(savepath,'clean_log.txt'),sep=' ',index=False)

    new_unc_tot = []

    for filter,filter_data in filter_split(data_ok).items():
        new_unc = np.array(flux_unc_val(filter_data))
        if filter != 'ZTF_i':
            new_unc_tot.append(new_unc)

        norm_jd = np.array(filter_data['jd'] - filter_data['jd'].iloc[0])
        clean_data = pd.DataFrame({"time":norm_jd,
                                    "flux":filter_data['forcediffimflux'].values,
                                    "flux_unc":new_unc,
                                    "zeropoint":filter_data['zpdiff'].values})
        
        if savepath != None:
            assert filename != None
            savestring = os.path.join(savepath,"clean_"+filter+"_"+filename)
            clean_data.to_csv(savestring,sep=' ',index=False)
        else:
            if verbose:
                print(filter,clean_data)
                print(20*'--')

    new_unc_tot = np.concatenate(new_unc_tot).ravel()
    clean_data_full['flux_unc'] = new_unc_tot

    if savepath != None:
        savestring = os.path.join(savepath,"clean_g_and_r"+filename)
        clean_data_full.to_csv(savestring,sep=' ',index=False)

In [158]:
##DIT MOET NOG GEUPDATET WORDEN:
"""
Het moet nu zo zijn dat clean data alle bad points er uit haalt en een file kan saven wat niet gesplit is op filters (behalve dat i er uit is).
Dan moet nog steeds de uncertainty validation gedaan worden en dat gaat filter gewijs. Vogel uit hoe je dit gaat doen. Verder wil je bij elk 
punt in de clean data dus hebben staan vanuit welke filter het komt (dit gaan we later nodig hebben voor fitten.)
In de log moet ook (per filter) de start en eind datum van de referentie (dit voor baseline meuk.)
"""

def clean_data(data,filename=None,savepath=None,verbose=False):
    iok = q_cuts(data) * field_check(data)
    idx_bad = np.where(np.invert(iok))[0]

    data_ok = data[iok]

    no_ir_idx = np.where(data_ok['filter'] != 'ZTF_i')[0]
    data_ok_no_ir = data_ok.copy(deep=True)
    data_ok_no_ir = data_ok_no_ir.iloc[no_ir_idx]
    try:
        norm_jd_full = np.array(data_ok_no_ir['jd'] - data_ok_no_ir['jd'].iloc[0])
    except:
        if iok.sum() == 0:
            print(f"{savepath}: no viable data found. Proceeding.")
            return 
        else:
            if 'ZTF_g' not in data_ok['filter'].unique() or 'ZTF_r' not in data_ok['filter'].unique():
                print("There is no viable ZTF_g or there is no viable ZTF_r data. Proceeding cleaning.")
                for filter,filter_data in filter_split(data_ok).items():
                    new_unc = np.array(flux_unc_val(filter_data))

                    norm_jd = np.array(filter_data['jd'] - filter_data['jd'].iloc[0])
                    clean_data = pd.DataFrame({"time":norm_jd,
                                                "flux":filter_data['forcediffimflux'].values,
                                                "flux_unc":new_unc,
                                                "zeropoint":filter_data['zpdiff'].values})
                    if savepath != None:
                        assert filename != None, 'Filename should be specified if savepath is specified.'
                        savestring = os.path.join(savepath,"clean_"+filter+"_"+filename)
                        clean_data.to_csv(savestring,sep=' ',index=False)
                    else:
                        if verbose:
                            print(filter,clean_data)
                            print(20*'--')
                return


    clean_data_full = pd.DataFrame({"time":norm_jd_full,"flux":data_ok_no_ir['forcediffimflux'].values,
                                # "flux_unc":np.zeros(data_ok_no_ir.shape[0]),
                                "zeropoint":data_ok_no_ir['zpdiff'].values,
                                "filter" :data_ok_no_ir['filter'].values})

    if savepath != None:
        pd.DataFrame({"bad_idx":[idx_bad],"median_chi_before":np.median(data['forcediffimchisq']),"median_chi_after":np.median(data_ok['forcediffimchisq'])}).to_csv(\
            os.path.join(savepath,'clean_log.txt'),sep=' ',index=False)

    new_unc_tot = []

    for filter,filter_data in filter_split(data_ok).items():
        new_unc = np.array(flux_unc_val(filter_data))
        if filter != 'ZTF_i':
            new_unc_tot.append(new_unc)

        norm_jd = np.array(filter_data['jd'] - filter_data['jd'].iloc[0])
        clean_data = pd.DataFrame({"time":norm_jd,
                                    "flux":filter_data['forcediffimflux'].values,
                                    "flux_unc":new_unc,
                                    "zeropoint":filter_data['zpdiff'].values})
        
        if savepath != None:
            assert filename != None
            savestring = os.path.join(savepath,"clean_"+filter+"_"+filename)
            clean_data.to_csv(savestring,sep=' ',index=False)
        else:
            if verbose:
                print(filter,clean_data)
                print(20*'--')

    new_unc_tot = np.concatenate(new_unc_tot).ravel()
    clean_data_full['flux_unc'] = new_unc_tot

    if savepath != None:
        savestring = os.path.join(savepath,"clean_g_and_r"+filename)
        clean_data_full.to_csv(savestring,sep=' ',index=False)

clean_data(test_lc,verbose=0)


In [156]:
sjoertpath = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\Sjoert_Flares'
sjoertflares = pd.read_csv(r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\ZTF_neoWISE_flares_acflares.dat',delimiter=' ')
catalog_coords = coord.SkyCoord(np.array(sjoertflares['ra']),np.array(sjoertflares['dec']),unit='deg')

for folder in os.listdir(sjoertpath)[::-1]:
    if "ZTF" in folder:
        folderpath = os.path.join(sjoertpath,folder)
        for file in os.listdir(folderpath):
            if 'clean' not in file:
                filepath = os.path.join(folderpath,file)
                test = np.genfromtxt(filepath,skip_header=53,dtype=dtypes)
                data = pd.DataFrame(np.genfromtxt(filepath,skip_header=53,dtype=dtypes))
                clean_data(data,filename=file,savepath=folderpath)



C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\Sjoert_Flares\ZTF19abisdrk: no viable data found. Proceeding.


In [157]:
for folder in os.listdir(sjoertpath)[::-1]:
    if "ZTF" in folder:
        folderpath = os.path.join(sjoertpath,folder)
        if len(os.listdir(folderpath)) < 5:
            if True not in ['clean_ZTF_i' in name for name in os.listdir(folderpath)]:
                print(folder,'first')
            print(folder,'second')
            print()

ZTF19abisdrk first
ZTF19abisdrk second

ZTF19aaohxwd first
ZTF19aaohxwd second



### Now performing the whole photometry business on the one with cleaned files.

In [125]:
def flux_jy(data):
    zp = sjoert.stellar.mag2flux(data['zeropoint'])
    flux = zp * data['flux'] * 1e6
    err = zp * data['flux_unc'] * 1e6
    return flux,err

def plot_clean_data(data,folder,file,savefig = False,verbose=False):
    flux,err = flux_jy(data)
    filter = file[10:][:1]
    plt.figure()
    plt.xlabel("time (jd)")
    plt.ylabel("Flux (mJy)")
    plt.title(folder + ' - ' + filter.upper() + " filter")
    plt.errorbar(data['time'],flux,err,fmt='.')
    if verbose and not savefig:
        plt.show()
    if savefig:
        plt.savefig(os.path.join(sjoertpath,"first_run_plots",folder+"_"+filter.upper()+" filter.png"),dpi=100)
    plt.close()

testdir = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\Sjoert_Flares\ZTF20aagkqky'
for folder in os.listdir(sjoertpath):
    if "ZTF" in folder:
        folderpath = os.path.join(sjoertpath,folder)
        for file in os.listdir(folderpath):
            if 'clean' in file and 'log' not in file:
                data = pd.read_csv(os.path.join(folderpath,file),delimiter=' ')
                plot_clean_data(data,folder,file,0,0)